<a href="https://colab.research.google.com/github/commando199/code_thesis/blob/master/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import os
from sklearn.metrics import confusion_matrix
import keras
import sklearn
from mpl_toolkits.mplot3d import Axes3D
import tempfile
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import *
import pandas as pd
import matplotlib as mpl
import re
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
import numpy as np
import seaborn as sns
from tensorflow.keras import *
import string 
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split


In [ ]:
#load dataset
#change path if needed
raw_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/global_idea_hunt_with_evals_no_smote.csv')
df = raw_df.copy()

In [ ]:
#train test val split (60,20,20) -> 0.8*0.25 = 0.6
X_train, X_test, y_train, y_test = train_test_split(df[['Visitors','Likes','No of Comments','Voters']], df['top_ideas'], stratify=df['top_ideas'],
                                                    test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1, stratify=y_train)




#ratio of good to bad ideas
neg, pos = np.bincount(y_train)
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))


weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = ((1 / pos) * (total / 2.0))

# calculate class weights
class_weight = {0: weight_for_0, 1: weight_for_1}

#print class weights
print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))




Visitors          int64
Likes             int64
No of Comments    int64
Voters            int64
dtype: object
(1773, 4)
      Visitors  Likes  No of Comments  Voters
2013         5      0               1       0
1878         6      1               2       0
374         38     17               9      11
420         35     19               7       9
414         36      1               2       0
...        ...    ...             ...     ...
1123        15     10               3       2
917         19      7               2       2
1196        14      2               2       1
1882         6      0               2       0
2027         5      0               0       0

[1773 rows x 4 columns]
Examples:
    Total: 1329
    Positive: 68 (5.12% of total)

Weight for class 0: 0.53
Weight for class 1: 9.77


In [ ]:
#metrics in order to evaluate performance
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]


In [ ]:
#building of the model
#at the moment a very basic and standard mlp
model = Sequential()
model.add(Dense(100, input_dim=4, activation='relu',
                kernel_initializer='random_normal'))
model.add(Dense(50,activation='relu',kernel_initializer='random_normal'))
model.add(Dense(25,activation='relu',kernel_initializer='random_normal'))
model.add(Dense(1,activation='sigmoid',kernel_initializer='random_normal'))

#compiling the model using binary crossentropy as a loss, adam as the optimizer and the metrics previously defined to see performance
model.compile(loss='binary_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(),
              metrics =METRICS)


#fitting model with our train and validation data sets (epochs are pretty random and no early stopping yet implemented), using the class weights previously calculated
model.fit(X_train,y_train,validation_data=(X_test,y_test),verbose=2,epochs=200,class_weight=class_weight)

      Visitors  Likes  No of Comments  Voters
1550        10      0               1       0
760         23      1               1       0
1533        10      4               1       0
255         48     26              21      12
64          77     30              12      33
...        ...    ...             ...     ...
142         60      6               2       2
1945         6      0               0       0
607         28      5               2       3
1871         6      1               3       2
60          78     31              12      16

[1329 rows x 4 columns]
1550    0
760     0
1533    0
255     0
64      0
       ..
142     1
1945    0
607     0
1871    0
60      0
Name: top_ideas, Length: 1329, dtype: int64
Epoch 1/500
42/42 - 15s - loss: 0.6887 - tp: 65.0000 - fp: 1162.0000 - tn: 99.0000 - fn: 3.0000 - accuracy: 0.1234 - precision: 0.0530 - recall: 0.9559 - auc: 0.5893 - prc: 0.0711 - val_loss: 0.7485 - val_tp: 23.0000 - val_fp: 418.0000 - val_tn: 3.0000 - val_fn: 0.0000

14/14 [==============================] - 0s 7ms/step - loss: 0.8066 - tp: 13.0000 - fp: 71.0000 - tn: 350.0000 - fn: 10.0000 - accuracy: 0.8176 - precision: 0.1548 - recall: 0.5652 - auc: 0.7332 - prc: 0.3573


ValueError: ignored

In [ ]:
#work in progress
history_dict = history.history
print(history_dict.keys())

acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# r is for "solid red line"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')